In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import requests, re
from lxml import etree
import xml.etree.ElementTree as ET
from tqdm import tqdm_notebook as tqdm
import zipfile, urllib
import pandas as pd
pd.set_option('display.max_columns', None)
from scipy import stats
import collections
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt
import nltk
import warnings
warnings.filterwarnings(action='once')

In [4]:
example_df = pd.read_csv("/content/drive/My Drive/NIH Projects/RePORTER_00_19.csv", converters={"project_terms": lambda x: x.strip("[]").replace("'","").split(", ")})

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (6,10,18,22,27,33,34,40,44,45) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
lop_list = []

for i, j in zip(example_df['project_start'], example_df['project_end']):
    
    try:
        date_diff = (dt.datetime.strptime(j, '%M/%d/%Y') - dt.datetime.strptime(i, '%M/%d/%Y')).days
        date_diff = date_diff / 365
    except TypeError:
        date_diff = ''
    
    lop_list.append(date_diff)

example_df['length_of_project'] = lop_list

In [7]:
criteria_1 = example_df['funding_mechanism'] == 'Non-SBIR/STTR RPGs'
criteria_2 = example_df['funding_mechanism'] == 'SBIR-STTR RPGs'
criteria_3 = example_df['funding_mechanism'] == 'SBIR-STTR'
criteria_4 = example_df['funding_mechanism'] == 'Research Projects'


criteria_all = criteria_1 | criteria_2 | criteria_3 | criteria_4

trimmed_df = example_df[criteria_all]

print(example_df.shape, trimmed_df.shape)

(1537644, 49) (467168, 49)


# Term Topic Modeling

In [8]:
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
nltk.download('stopwords')

terms = list(trimmed_df['project_terms'])

big_fin = []

example_sent = terms[0]
  
stop_words = set(stopwords.words('english'))
new_words = ["base", "phase", "improved", "data", "design", "testing", "development", "goals", "novel", "public health relevance"]
stop_words = stop_words.union(new_words) 

for i in terms:
  fin = []  
  for w in i: 
      w = w.lower()
      if w not in stop_words: 
          fin.append(w)
  big_fin.append(fin)
  
print(terms[0:3]) 
print(big_fin[0:3]) 

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


/usr/local/lib/python3.6/dist-packages/nltk/corpus/reader/wordlist.py:28: ResourceWarning: unclosed file <_io.BufferedReader name='/root/nltk_data/corpora/stopwords/english'>
  return concat([self.open(f).read() for f in fileids])


[['Adhesives', 'Animal Model', 'Antibodies', 'Cell physiology', 'Cells', 'Cellular biology', 'Disease', 'Engineering', 'Event', 'HIV', 'HIV Infections', 'Hour', 'Image', 'in vivo', 'Infection', 'innovation', 'Modeling', 'Molecular Cloning', 'Movement', 'Mus', 'novel', 'Process', 'Relative (related person)', 'Route', 'Structure', 'Synapses', 'synaptogenesis', 'T-Lymphocyte', 'Testing', 'transmission process', 'Viral', 'Virion', 'virological synapse', 'Virus', 'Virus-Cell Membrane Interaction'], ['analog', 'Antiviral Agents', 'Brain', 'cell type', 'Cells', 'cost', 'Development', 'DNA Methylation', 'DNA Methyltransferase', 'DNA Modification Methylases', 'Double-Stranded RNA', 'drug development', 'drug of abuse', 'Epigenetic Process', 'Face', 'Gene Silencing', 'Genes', 'Genetic Transcription', 'Highly Active Antiretroviral Therapy', 'Histone Acetylation', 'HIV', 'Individual', 'latent infection', 'Lead', 'Life', 'macrophage', 'Mediator of activation protein', 'memory CD4 T lymphocyte', 'Mic

In [9]:
from gensim import corpora
dictionary = corpora.Dictionary(big_fin)
corpus = [dictionary.doc2bow(text) for text in big_fin]

import pickle
pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/usr/local/lib/python3.6/dist-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: ResourceWarning: unclosed file <_io.BufferedWriter name='corpus.pkl'>
  
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:40

In [10]:
import gensim
NUM_TOPICS = 10
ldamodel10 = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
ldamodel10.save('model10.gensim')
topics = ldamodel.print_topics(num_words=10)
for topic in topics:
    print(topic)

/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)


KeyboardInterrupt: ignored

In [0]:
NUM_TOPICS = 20
ldamodel20 = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
ldamodel20.save('model20.gensim')
topics = ldamodel.print_topics(num_words=10)
for topic in topics:
    print(topic)

/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)


In [0]:
"""dictionary = gensim.corpora.Dictionary.load('dictionary.gensim')
corpus = pickle.load(open('corpus.pkl', 'rb'))
lda = gensim.models.ldamodel.LdaModel.load('model5.gensim')

lda_display = pyLDAvis.gensim.prepare(lda, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)"""

In [0]:
# Show Topics
pprint(ldamodel20.show_topics(formatted=False))

# Compute Coherence Score
coherence_model_ldamodel20 = CoherenceModel(model=ldamodel20, texts=big_fin, dictionary=dictionary, coherence='c_v')
coherence_ldamodel20 = coherence_model_ldamodel20.get_coherence()
print('\nCoherence Score: ', coherence_ldamodel20)

In [0]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [0]:
# Can take a long time to run.
model_list, coherence_values = compute_coherence_values(dictionary=dictionary, corpus=corpus, texts=big_fin, start=10, limit=50, step=5)

In [12]:
# Show graph
limit=40; start=2; step=6;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

NameError: ignored

In [13]:
# Print the coherence scores
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))

NameError: ignored

In [0]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(ldamodel20, corpus, id2word)
vis